<h1>Data Science Capstone</h1>

<i>Ronald R. Rodriguez</i>

This project serves as the capstone project for the capstone course given by IBM. This project serves to use the skills learned studying for the IBM Data Science Ceritification in a case study.



In [13]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import io
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


In [4]:
CLIENT_ID = 'Secret' # your Foursquare ID
CLIENT_SECRET = 'Secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Secret
CLIENT_SECRET:Secret


Here is an example of NYC station longitude and latitudes

In [67]:
station_csv = requests.Session().get('http://web.mta.info/developers/data/nyct/subway/Stations.csv').content.decode('UTF-8')
station_data = pd.read_csv(io.StringIO(station_csv))
station_loc = station_data[["Stop Name","Borough","GTFS Latitude","GTFS Longitude"]]
print(station_loc.head())
print('new york station data collected')

                Stop Name Borough  GTFS Latitude  GTFS Longitude
0  Astoria - Ditmars Blvd       Q      40.775036      -73.912034
1            Astoria Blvd       Q      40.770258      -73.917843
2                   30 Av       Q      40.766779      -73.921479
3                Broadway       Q      40.761820      -73.925508
4                   36 Av       Q      40.756804      -73.929575
new york station data collected


Here is an example of SF Bart station latitude/longitude and names

In [66]:
station_json = requests.get('http://api.bart.gov/api/stn.aspx?cmd=stns&key=MW9S-E7SL-26DU-VV8V&json=y').json()
station_data = pd.DataFrame.from_dict(station_json['root']['stations']['station'])
station_loc = station_data[["name","city","gtfs_latitude","gtfs_longitude"]]
print(station_loc.head())
print('San Fran station data collected')

                           name           city gtfs_latitude gtfs_longitude
0  12th St. Oakland City Center        Oakland     37.803768    -122.271450
1              16th St. Mission  San Francisco     37.765062    -122.419694
2              19th St. Oakland        Oakland     37.808350    -122.268602
3              24th St. Mission  San Francisco     37.752470    -122.418143
4                       Antioch        Antioch     37.995388    -121.780420
San Fran station data collected


Here is an example of Austin Station latitude/longitude and names

In [70]:
station_csv = requests.Session().get('https://transitfeeds-data.s3-us-west-1.amazonaws.com/public/feeds/capital-metro/24/20181218/original/stops.txt').content.decode('UTF-8')
station_data = pd.read_csv(io.StringIO(station_csv))
station_loc = station_data[["stop_name","stop_lat","stop_lon"]]
print(station_loc.head())
print('Austin station data collected')

               stop_name   stop_lat   stop_lon
0        Rio Grande/15th  30.278966 -97.746339
1  8740 Lamar/Payton Gin  30.356809 -97.701066
2   8630 Lamar/Fairfield  30.355296 -97.703128
3         Lamar/Thurmond  30.353139 -97.706082
4  8320 Lamar/Meadowlark  30.352256 -97.707295
Austin station data collected


With the station data it can be combined get a diverse list of stations 
to use so that there is less bias coming from location.

Then using the foursquare app get the nearest venues from those locations. Then classify the station by the most prevalent category then use ml to analyze whether there is a corelation between station location and category. Here is an example using the foursquare api using the first station from NYC, Astoria-Ditmars Blvd.

In [92]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 40.775036, -73.912034, 20180604, 500, 35)
venue_json = requests.get(url).json()
#print(venue_json["response"]["venues"])
venue_data = pd.DataFrame.from_dict(venue_json["response"]["venues"])
print(venue_data["categories"])

0     [{'name': 'Metro Station', 'pluralName': 'Metr...
1     [{'name': 'Deli / Bodega', 'pluralName': 'Deli...
2     [{'name': 'Middle Eastern Restaurant', 'plural...
3     [{'name': 'Supermarket', 'pluralName': 'Superm...
4     [{'name': 'Miscellaneous Shop', 'pluralName': ...
5     [{'name': 'Pub', 'pluralName': 'Pubs', 'shortN...
6     [{'name': 'Bagel Shop', 'pluralName': 'Bagel S...
7     [{'name': 'Bookstore', 'pluralName': 'Bookstor...
8     [{'name': 'General Entertainment', 'pluralName...
9     [{'name': 'Turkish Restaurant', 'pluralName': ...
10    [{'name': 'Bike Rental / Bike Share', 'pluralN...
11    [{'name': 'Fried Chicken Joint', 'pluralName':...
12    [{'name': 'Convenience Store', 'pluralName': '...
13    [{'name': 'Diner', 'pluralName': 'Diners', 'sh...
14    [{'name': 'Yoga Studio', 'pluralName': 'Yoga S...
15    [{'name': 'Fast Food Restaurant', 'pluralName'...
16    [{'name': 'Shoe Store', 'pluralName': 'Shoe St...
17    [{'name': 'Bank', 'pluralName': 'Banks', '

For this example 'Yoga Studio' appears twice so this as test data would be labeled as 'Yoga Studio'.